# Get Data from the Census API

This notebook shows how to get data from the U.S. Census Bureau API, and construct a block group-level dataset using publicly available American Community Survey 5-year data.

In [1]:
import requests
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def get_census_data(year, dataset, variables, location):
    host = 'https://api.census.gov/data'
    year = f'/{year}'
    dataset = f'/{dataset}'
    get = '?get='
    variables = f'{variables}'
    location = f'&for={location}'
    url = f'{host}{year}{dataset}{get}{variables}{location}'
    response = requests.get(url)
    return response

In [4]:
def json_to_dataframe(response):
    df = pd.DataFrame(response.json()[1:], columns=response.json()[0])
    return df

In [5]:
def get_data_dictionary(year, dataset):
    url = f'https://api.census.gov/data/{year}/{dataset}/variables.html'
    df = pd.read_html(url)
    df = pd.DataFrame(df[0])
    df['Label'].replace({"!!":" ", ":":""}, regex=True, inplace=True)
    return df

In [6]:
def calculate_percents(df, denominator, numerators):
    df[denominator] = df[denominator].astype(float)
    for i in numerators:
        df[i] = df[i].astype(float)
        newname = f'{i}_P'
        df[newname] = df[i] / df[denominator]

Define parameters of interest. We will look at American Community Survey 2021 5-Year estimates for all census block groups in California.

In [7]:
year = '2021'
dataset = 'acs/acs5'
location = 'block%20group:*&in=state:06&in=county:*&in=tract:*'

Get American Community Survey data for selected topics.

In [8]:
census_data = get_census_data(year=year,dataset=dataset,variables='GEO_ID',location=location)
census_data = json_to_dataframe(census_data)

pop = 'B01003_001E'
age = 'B01002_001E'
sex = 'B01001_001E,B01001_002E,B01001_026E'
race = 'B03002_001E,B03002_002E,B03002_003E,B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_012E'
lang = 'B99162_001E,B99162_002E,B99162_003E'
inc = 'B19013_001E'
pov = 'C17002_001E,C17002_002E,C17002_003E,C17002_004E,C17002_005E,C17002_006E,C17002_007E,C17002_008E'
mob = 'B07201_001E,B07201_002E,B07201_003E,B07201_014E'
trans = 'B08301_001E,B08301_002E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E'
hous = 'B25003_001E,B25003_002E,B25003_003E'

for i in [pop,age,sex,race,lang,inc,pov,mob,trans,hous]:
    i = 'GEO_ID,' + i
    temp = get_census_data(year=year,dataset=dataset,variables=i,location=location)
    temp = json_to_dataframe(temp)
    temp.drop(columns=['state', 'county', 'tract', 'block group'], inplace=True)
    census_data = census_data.merge(temp, on='GEO_ID', how='left')

census_data.head(3)

,GEO_ID,state,county,tract,block group,B01003_001E,B01002_001E,B01001_001E,B01001_002E,B01001_026E,B03002_001E,B03002_002E,B03002_003E,B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_012E,B99162_001E,B99162_002E,B99162_003E,B19013_001E,C17002_001E,C17002_002E,C17002_003E,C17002_004E,C17002_005E,C17002_006E,C17002_007E,C17002_008E,B07201_001E,B07201_002E,B07201_003E,B07201_014E,B08301_001E,B08301_002E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E,B25003_001E,B25003_002E,B25003_003E
0,1500000US060014001001,06,001,400100,1,1963,47.1,1963,1054,909,1963,1847,1429,44,0,257,0,17,100,116,1940,1602,338,190179,1963,29,41,83,37,33,20,1720,1950,1770,180,0,938,435,89,0,0,0,102,0,312,699,599,100
1,1500000US060014001002,06,001,400100,2,1361,55.7,1361,596,765,1361,1227,904,60,0,221,0,0,42,134,1263,772,491,233125,1350,87,16,0,15,0,0,1232,1361,1180,143,38,731,385,112,0,0,0,0,10,224,638,575,63
2,1500000US060014002001,06,001,400200,1,1029,46.9,1029,487,542,1029,959,817,6,0,47,0,0,89,70,945,850,95,232000,1029,0,24,0,25,0,0,980,1023,925,98,0,543,202,133,0,10,0,22,0,176,456,319,137


Relabel the columns with more meaningful names.

In [9]:
dictionary = get_data_dictionary('2021', 'acs/acs5')
dictionary.set_index('Name', inplace=True)

for i in census_data.columns:
    if i in dictionary.index:
        temp = dictionary[dictionary.index == i]
        newname = temp['Label'].iloc[0]
        newname = newname.replace(' ', '_')
        census_data = census_data.rename(columns={i:f'{i}_{newname}'})

census_data.head(3)

,GEO_ID_Geography,state,county,tract,block group,B01003_001E_Estimate_Total,B01002_001E_Estimate_Median_age_--_Total,B01001_001E_Estimate_Total,B01001_002E_Estimate_Total_Male,B01001_026E_Estimate_Total_Female,B03002_001E_Estimate_Total,B03002_002E_Estimate_Total_Not_Hispanic_or_Latino,B03002_003E_Estimate_Total_Not_Hispanic_or_Latino_White_alone,B03002_004E_Estimate_Total_Not_Hispanic_or_Latino_Black_or_African_American_alone,B03002_005E_Estimate_Total_Not_Hispanic_or_Latino_American_Indian_and_Alaska_Native_alone,B03002_006E_Estimate_Total_Not_Hispanic_or_Latino_Asian_alone,B03002_007E_Estimate_Total_Not_Hispanic_or_Latino_Native_Hawaiian_and_Other_Pacific_Islander_alone,B03002_008E_Estimate_Total_Not_Hispanic_or_Latino_Some_other_race_alone,B03002_009E_Estimate_Total_Not_Hispanic_or_Latino_Two_or_more_races,B03002_012E_Estimate_Total_Hispanic_or_Latino,B99162_001E_Estimate_Total,B99162_002E_Estimate_Total_Speak_only_English,B99162_003E_Estimate_Total_Speak_other_languages,B19013_001E_Estimate_Median_household_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars),C17002_001E_Estimate_Total,C17002_002E_Estimate_Total_Under_.50,C17002_003E_Estimate_Total_.50_to_.99,C17002_004E_Estimate_Total_1.00_to_1.24,C17002_005E_Estimate_Total_1.25_to_1.49,C17002_006E_Estimate_Total_1.50_to_1.84,C17002_007E_Estimate_Total_1.85_to_1.99,C17002_008E_Estimate_Total_2.00_and_over,B07201_001E_Estimate_Total,B07201_002E_Estimate_Total_Same_house_1_year_ago,B07201_003E_Estimate_Total_Different_house_in_United_States_1_year_ago,B07201_014E_Estimate_Total_Abroad_1_year_ago,B08301_001E_Estimate_Total,"B08301_002E_Estimate_Total_Car,_truck,_or_van",B08301_010E_Estimate_Total_Public_transportation_(excluding_taxicab),B08301_016E_Estimate_Total_Taxicab,B08301_017E_Estimate_Total_Motorcycle,B08301_018E_Estimate_Total_Bicycle,B08301_019E_Estimate_Total_Walked,B08301_020E_Estimate_Total_Other_means,B08301_021E_Estimate_Total_Worked_from_home,B25003_001E_Estimate_Total,B25003_002E_Estimate_Total_Owner_occupied,B25003_003E_Estimate_Total_Renter_occupied
0,1500000US060014001001,06,001,400100,1,1963,47.1,1963,1054,909,1963,1847,1429,44,0,257,0,17,100,116,1940,1602,338,190179,1963,29,41,83,37,33,20,1720,1950,1770,180,0,938,435,89,0,0,0,102,0,312,699,599,100
1,1500000US060014001002,06,001,400100,2,1361,55.7,1361,596,765,1361,1227,904,60,0,221,0,0,42,134,1263,772,491,233125,1350,87,16,0,15,0,0,1232,1361,1180,143,38,731,385,112,0,0,0,0,10,224,638,575,63
2,1500000US060014002001,06,001,400200,1,1029,46.9,1029,487,542,1029,959,817,6,0,47,0,0,89,70,945,850,95,232000,1029,0,24,0,25,0,0,980,1023,925,98,0,543,202,133,0,10,0,22,0,176,456,319,137


Convert counts into percentages. Depending on the item, the value of the denominator may be different so it's important to use the correct denominator for each feature.

In [10]:
prefixes = []
for i in census_data.columns:
    prefix = i[:6]
    if prefix not in prefixes and prefix not in ['GEO_ID', 'state', 'county', 'tract', 'block ']:
        prefixes = prefixes + [prefix]
        
for i in prefixes:
    denominator = f'{i}_001E_Estimate_Total'
    numerators = [col for col in census_data if col.startswith(i)]
    if denominator in numerators:
        numerators.remove(denominator)
    if denominator in census_data.columns and len(numerators) >= 1:
        calculate_percents(df=census_data, denominator=denominator, numerators=numerators)

census_data.head(3)

,GEO_ID_Geography,state,county,tract,block group,B01003_001E_Estimate_Total,B01002_001E_Estimate_Median_age_--_Total,B01001_001E_Estimate_Total,B01001_002E_Estimate_Total_Male,B01001_026E_Estimate_Total_Female,B03002_001E_Estimate_Total,B03002_002E_Estimate_Total_Not_Hispanic_or_Latino,B03002_003E_Estimate_Total_Not_Hispanic_or_Latino_White_alone,B03002_004E_Estimate_Total_Not_Hispanic_or_Latino_Black_or_African_American_alone,B03002_005E_Estimate_Total_Not_Hispanic_or_Latino_American_Indian_and_Alaska_Native_alone,B03002_006E_Estimate_Total_Not_Hispanic_or_Latino_Asian_alone,B03002_007E_Estimate_Total_Not_Hispanic_or_Latino_Native_Hawaiian_and_Other_Pacific_Islander_alone,B03002_008E_Estimate_Total_Not_Hispanic_or_Latino_Some_other_race_alone,B03002_009E_Estimate_Total_Not_Hispanic_or_Latino_Two_or_more_races,B03002_012E_Estimate_Total_Hispanic_or_Latino,B99162_001E_Estimate_Total,B99162_002E_Estimate_Total_Speak_only_English,B99162_003E_Estimate_Total_Speak_other_languages,B19013_001E_Estimate_Median_household_income_in_the_past_12_months_(in_2021_inflation-adjusted_dollars),C17002_001E_Estimate_Total,C17002_002E_Estimate_Total_Under_.50,C17002_003E_Estimate_Total_.50_to_.99,C17002_004E_Estimate_Total_1.00_to_1.24,C17002_005E_Estimate_Total_1.25_to_1.49,C17002_006E_Estimate_Total_1.50_to_1.84,C17002_007E_Estimate_Total_1.85_to_1.99,C17002_008E_Estimate_Total_2.00_and_over,B07201_001E_Estimate_Total,B07201_002E_Estimate_Total_Same_house_1_year_ago,B07201_003E_Estimate_Total_Different_house_in_United_States_1_year_ago,B07201_014E_Estimate_Total_Abroad_1_year_ago,B08301_001E_Estimate_Total,"B08301_002E_Estimate_Total_Car,_truck,_or_van",B08301_010E_Estimate_Total_Public_transportation_(excluding_taxicab),B08301_016E_Estimate_Total_Taxicab,B08301_017E_Estimate_Total_Motorcycle,B08301_018E_Estimate_Total_Bicycle,B08301_019E_Estimate_Total_Walked,B08301_020E_Estimate_Total_Other_means,B08301_021E_Estimate_Total_Worked_from_home,B25003_001E_Estimate_Total,B25003_002E_Estimate_Total_Owner_occupied,B25003_003E_Estimate_Total_Renter_occupied,B01001_002E_Estimate_Total_Male_P,B01001_026E_Estimate_Total_Female_P,B03002_002E_Estimate_Total_Not_Hispanic_or_Latino_P,B03002_003E_Estimate_Total_Not_Hispanic_or_Latino_White_alone_P,B03002_004E_Estimate_Total_Not_Hispanic_or_Latino_Black_or_African_American_alone_P,B03002_005E_Estimate_Total_Not_Hispanic_or_Latino_American_Indian_and_Alaska_Native_alone_P,B03002_006E_Estimate_Total_Not_Hispanic_or_Latino_Asian_alone_P,B03002_007E_Estimate_Total_Not_Hispanic_or_Latino_Native_Hawaiian_and_Other_Pacific_Islander_alone_P,B03002_008E_Estimate_Total_Not_Hispanic_or_Latino_Some_other_race_alone_P,B03002_009E_Estimate_Total_Not_Hispanic_or_Latino_Two_or_more_races_P,B03002_012E_Estimate_Total_Hispanic_or_Latino_P,B99162_002E_Estimate_Total_Speak_only_English_P,B99162_003E_Estimate_Total_Speak_other_languages_P,C17002_002E_Estimate_Total_Under_.50_P,C17002_003E_Estimate_Total_.50_to_.99_P,C17002_004E_Estimate_Total_1.00_to_1.24_P,C17002_005E_Estimate_Total_1.25_to_1.49_P,C17002_006E_Estimate_Total_1.50_to_1.84_P,C17002_007E_Estimate_Total_1.85_to_1.99_P,C17002_008E_Estimate_Total_2.00_and_over_P,B07201_002E_Estimate_Total_Same_house_1_year_ago_P,B07201_003E_Estimate_Total_Different_house_in_United_States_1_year_ago_P,B07201_014E_Estimate_Total_Abroad_1_year_ago_P,"B08301_002E_Estimate_Total_Car,_truck,_or_van_P",B08301_010E_Estimate_Total_Public_transportation_(excluding_taxicab)_P,B08301_016E_Estimate_Total_Taxicab_P,B08301_017E_Estimate_Total_Motorcycle_P,B08301_018E_Estimate_Total_Bicycle_P,B08301_019E_Estimate_Total_Walked_P,B08301_020E_Estimate_Total_Other_means_P,B08301_021E_Estimate_Total_Worked_from_home_P,B25003_002E_Estimate_Total_Owner_occupied_P,B25003_003E_Estimate_Total_Renter_occupied_P
0,1500000US060014001001,06,001,400100,1,1963,47.1,1963.0,1054.0,909.0,1963.0,1847.0,1429.0,44.0,0.0,257.0,0.0,17.0,100.0,116.0,1940.0,1602.0,338.0,190179,1963.0,29.0,41.0,83.0,37.0,33.0,20.0,1720.0,1950.0

Save the data.

In [11]:
census_data.to_csv('census_data.csv', index=False)

### Useful Resources

https://www.census.gov/data/developers/data-sets.html

https://www.census.gov/library/video/2020/using-api-all-results-for-acs-table.html

https://www.youtube.com/watch?v=l47HptzM7ao

https://medium.com/@mcmanus_data_works/using-the-u-s-census-bureau-api-with-python-5c30ad34dbd7

https://www.youtube.com/watch?v=LW-M_UC0VTE